In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [2]:
from pymongo import MongoClient
from pprint import pprint

In [3]:
import time

In [4]:
def initDB(name):
    client = MongoClient('localhost', 27017)
    return client[name]

In [5]:
def saveRow(row, client):
    collection = client['mail']
    collection.insert_one(row)

In [6]:
def printRows(client, limit=10):
    collection = client['mail']
    for item in collection.find().limit(limit):
        pprint(item)

In [7]:
def login(driver):
    elem = driver.find_element_by_id("mailbox:login")
    elem.clear()
    elem.send_keys('study.ai_172@mail.ru')
    elem.send_keys(Keys.RETURN)
    time.sleep(1)
    elem = driver.find_element_by_id("mailbox:password")
    elem.clear()
    elem.send_keys('NewPassword172')
    elem.send_keys(Keys.RETURN)

In [8]:
def saveLetter(driver, client):
    author = driver.find_element_by_xpath("//div[@class='letter__author']/span").get_attribute('title')
    date = driver.find_element_by_xpath("//div[@class='letter__date']").text
    subject = driver.find_element_by_class_name('thread__subject').text
    body = driver.find_element_by_xpath("//div[@class='letter-body']").text
    row = {'author': author, 'date': date, 'subject': subject, 'body': body}
    saveRow(row, client)

In [9]:
client = initDB('selenium')

In [18]:
chrome_options = Options()
#chrome_options.add_argument('start-maximized')
#chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://mail.ru/')

In [19]:
login(driver)

In [20]:
#llc js-tooltip-direction_letter-bottom js-letter-list-item llc_pony-mode llc_normal
first = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((
        By.CLASS_NAME, 'js-letter-list-item'
    ))
)
driver.get(first.get_attribute('href'))

In [21]:
driver.set_window_size(1080,1080)
while True:
    button_next = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((
                By.CLASS_NAME, 'button2_arrow-down'
            ))
        )
    try:
        saveLetter(driver, client)
    except Exception as e:
        print(e)
    if button_next.get_attribute('disabled'):
        break
    button_next.click()

In [25]:
printRows(client, 3)

{'_id': ObjectId('5e2cda1f65cbb84caddb4e18'),
 'author': 'security@e.mail.ru',
 'body': '          Вход с нового устройства в аккаунт\n'
         ' study.ai_172@mail.ru \n'
         '      В аккаунт study.ai_172@mail.ru вошли с нового устройства.\n'
         '  Если вы этого не делали, измените пароль, чтобы обезопасить '
         'аккаунт.\n'
         '  Время 26 января, 00:38\n'
         '  Устройство Microsoft Windows 10, Яндекс.Браузер, Москва, Россия\n'
         '  Посмотреть список устройств\n'
         '  Узнайте, как защитить аккаунт, на нашем сайте по безопасности.\n'
         '    Вы получили это письмо, потому что являетесь пользователем '
         'Сервиса Почта Mail\u200c.ru на основании Пользовательского '
         'соглашения.\n'
         '  Copyright 2020 Mail\u200c.ru Group, Москва — Все права защищены.\n'
         '  Служба поддержки\n'
         '   ',
 'date': 'Сегодня, 0:38',
 'subject': 'Вход с нового устройства в аккаунт'}
{'_id': ObjectId('5e2cda2065cbb84caddb4e1

In [24]:
driver.quit()